<a href="https://colab.research.google.com/github/OmdenaAI/omdena-bangladesh-roadsafety/blob/Task-2-Data_preprocessing_and_EDA/geocoding_using_pgeocode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### First, let's import all the required libraries and modules and install geopy to use it. We will geocode our locations using geopy

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)

import numpy as np

!pip install geopy -U

from geopy.geocoders import Nominatim

     |████████████████████████████████| 112kB 15.3MB/s 
  Found existing installation: geopy 1.17.0
    Uninstalling geopy-1.17.0:
      Successfully uninstalled geopy-1.17.0


####Mounting the .csv file with locations from my google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


####Using pandas to read the csv file

In [ ]:
data= pd.read_csv('/content/drive/MyDrive/locations.csv')

In [ ]:
data.head(10)

,Location
0,Madaripur
1,Satkhira
2,Cumilla
3,Jessore
4,"Bhaluka, mymensingh"
5,"Ukhia, Cox's Bazar"
6,"Ukhia, Cox's Bazar"
7,"Mirsarai, Chittagong"
8,"Ghorashal-Tongi road,Narsingdi"
9,"Shivalaya, Manikganj"


####Splitting our locations by comma into separate columns, then stripping off the whitespaces and replacing the NaN values with empty string

In [ ]:
data[['loc1','loc2','loc3']]=data['Location'].str.split(',', expand= True)

In [ ]:
data= data.applymap(lambda x: x.strip() if isinstance(x,str) else x)
data = data.replace(np.nan, '', regex=True)
data.head(10)

,Location,loc1,loc2,loc3
0,Madaripur,Madaripur,,
1,Satkhira,Satkhira,,
2,Cumilla,Cumilla,,
3,Jessore,Jessore,,
4,"Bhaluka, mymensingh",Bhaluka,mymensingh,
5,"Ukhia, Cox's Bazar",Ukhia,Cox's Bazar,
6,"Ukhia, Cox's Bazar",Ukhia,Cox's Bazar,
7,"Mirsarai, Chittagong",Mirsarai,Chittagong,
8,"Ghorashal-Tongi road,Narsingdi",Ghorashal-Tongi road,Narsingdi,
9,"Shivalaya, Manikganj",Shivalaya,Manikganj,


####Sample example for how to get geocodes using geopy

In [ ]:
#Sample example with geopy
geolocator= Nominatim(user_agent="new_application")
location = geolocator.geocode(f"Taltola	Satkhira Bangladesh")
location.latitude, location.longitude

(22.7166509, 89.0749857)

####Making a function for using geopy for getting geocodes

In [ ]:
def geocoding(query):
  geolocator= Nominatim(user_agent="new_application")
  location = geolocator.geocode(query)
  if location!=None:
    dic={'latitudes':location.latitude, 'longitudes':location.longitude}
    return dic

####As we have three columns separated, if we do not get any result using all three, then we will look for results with the last two, if still we don't get anything, then we will look for results for the last column only. We are adding 'Bangladesh' at the end for searching for locations only in Bangladesh

In [ ]:
def queries():
  result=[]
  for i in range(data.shape[0]):
    query1= data['loc1'][i]+' '+data['loc2'][i]+' '+data['loc3'][i]+' '+'Bangladesh'
    query2= data['loc2'][i]+' '+data['loc3'][i]+' '+'Bangladesh'
    query3= data['loc3'][i]+' '+'Bangladesh'
    codes= geocoding(query1)
    if codes==None:
      codes=geocoding(query2)
      if codes==None:
        codes= geocoding(query3)
        if codes==None:
          dic={'latitudes':'Not found', 'longitudes':'Not found'}
          result.append(dic)  
        else:
          result.append(codes)
      else:
        result.append(codes)
    else:
      result.append(codes)
  return result

In [ ]:
geocodes= queries()

In [ ]:
data= pd.concat([data,pd.DataFrame(geocodes)], axis=1)

####Dropping the rows for which we did not receive any geocode

In [ ]:
data= data[data['latitudes']!='Not found']

In [ ]:
data

,Location,loc1,loc2,loc3,latitudes,longitudes
0,Madaripur,Madaripur,,,23.167547,90.204973
1,Satkhira,Satkhira,,,22.716651,89.074986
2,Cumilla,Cumilla,,,23.461061,91.180875
3,Jessore,Jessore,,,23.084244,89.124694
4,"Bhaluka, mymensingh",Bhaluka,mymensingh,,24.404455,90.386354
...,...,...,...,...,...,...
262,"Mohammadpur, Magura",Mohammadpur,Magura,,23.440030,89.510217
263,"Kutukchari, Rangamati",Kutukchari,Rangamati,,22.817729,92.219707
264,"Faridganj, Chandpur",Faridganj,Chandpur,,23.128294,90.754796
265,"Jhikargacha, Jashore",Jhikargacha,Jashore,,23.104114,89.107623


####Exporting the .csv file to local machine

In [ ]:
from google.colab import files
data.to_csv('Geocodes_for_heatmap.csv', encoding='utf-8', index= False)
files.download("Geocodes_for_heatmap.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

####Now I will use these geocodes to make a heatmap using mapbox since mapbox doesn't require any kind of API keys